In [1]:
# Import the Pandas library

import openpyxl
import io
import tempfile
import pandas as pd
import numpy as np
import sys

sys.path.append('../')

from models.linear_regression import LinReg

lin_reg = LinReg()
path = "../.test_data/more_data.xlsx"

coded_df = lin_reg.process_excel(path)


print(coded_df.info())
coded_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43384 entries, 0 to 43383
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   NAME             43384 non-null  object        
 1   date             43384 non-null  datetime64[ns]
 2   color_and_value  43384 non-null  object        
 3   day_year         43384 non-null  int64         
 4   name_code        43384 non-null  int64         
 5   value            43384 non-null  float64       
 6   color            43384 non-null  object        
 7   color_code       43384 non-null  int64         
 8   next_color_code  43248 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 3.0+ MB
None


,NAME,date,color_and_value,day_year,name_code,value,color,color_code,next_color_code
0,GJ1,2024-12-28 00:00:00.000,19.1379 | FFFF00,363,1,19.137900,FFFFFF,0,1.0
1,GJ2,2024-12-28 00:00:00.000,44.45609 | FFFF00,363,2,44.456090,FFFFFF,0,1.0
2,GJ3,2024-12-28 00:00:00.000,65.52176 | A9D08E,363,3,65.521760,FFFFFF,0,0.0
3,GJ4,2024-12-28 00:00:00.000,28.03879 | FFFF00,363,4,28.038790,FFFFFF,0,0.0
4,GJ5,2024-12-28 00:00:00.000,12.66648 | 9BC2E6,363,5,12.666480,000000,1,1.0
...,...,...,...,...,...,...,...,...,...
43379,GJ132,2024-04-12 00:00:00.100,-10.511 | 8EA9DB,103,132,-10.511000,FFFFFF,0,NaN
43380,GJ133,2024-04-12 00:00:00.100,2.78895 | 8EA9DB,103,133,2.788950,FFFFFF,0,NaN
43381,GJ134,2024-04-12 00:00:00.100,4.872244 | FFC000,103,134,4.872244,000000,1,NaN
43382,GJ135,2024-04-12 00:00:00.100,78.84801 | FFC000,103,135,78.848010,000000,1,NaN


In [2]:
lin_reg.color_mapping


{0: 'FFFFFF', 1: '000000'}

In [3]:
coded_df[ coded_df['color'] == 'D5A6BD']
# 46376    484 purple    2382 orange

,NAME,date,color_and_value,day_year,name_code,value,color,color_code,next_color_code


In [4]:
lin_reg.last_df

,NAME,2024-04-12 00:00:00.1
0,GJ1,-6.18032 | FFC000
1,GJ2,-4.03207 | 8EA9DB
2,GJ3,-15.2723 | FFFF00
3,GJ4,-16.9018 | A9D08E
4,GJ5,-5.7277 | A9D08E
...,...,...
131,GJ132,-10.511 | 8EA9DB
132,GJ133,2.78895 | 8EA9DB
133,GJ134,4.872244 | FFC000
134,GJ135,78.84801 | FFC000


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split



# Splitting the dataset into the Training set and Test set
X , y = lin_reg.train_test_split(coded_df)


X.shape, y.shape

((43248, 4), (43248,))

In [6]:
# Create and train the model


lin_reg.fit(X, y)

print(f'mse : {lin_reg.mse}' )

print(f'accuracy : {lin_reg.accuracy}' )
print(f'preferred accuracy: {lin_reg.preferred_accuracy}' )

[1]
mse : 0.3668208092485549
accuracy : 0.6331791907514451
preferred accuracy: 0.51875


In [7]:
lin_reg.last_df

,NAME,2024-04-12 00:00:00.1
0,GJ1,-6.18032 | FFC000
1,GJ2,-4.03207 | 8EA9DB
2,GJ3,-15.2723 | FFFF00
3,GJ4,-16.9018 | A9D08E
4,GJ5,-5.7277 | A9D08E
...,...,...
131,GJ132,-10.511 | 8EA9DB
132,GJ133,2.78895 | 8EA9DB
133,GJ134,4.872244 | FFC000
134,GJ135,78.84801 | FFC000


In [8]:
# make predictions
df = lin_reg.predict_last()

df

,NAME,2024-04-12 00:00:00.1,next_color_code,next_color
0,GJ1,-6.18032 | FFC000,0,FFFFFF
1,GJ2,-4.03207 | 8EA9DB,0,FFFFFF
2,GJ3,-15.2723 | FFFF00,0,FFFFFF
3,GJ4,-16.9018 | A9D08E,0,FFFFFF
4,GJ5,-5.7277 | A9D08E,0,FFFFFF
...,...,...,...,...
131,GJ132,-10.511 | 8EA9DB,0,FFFFFF
132,GJ133,2.78895 | 8EA9DB,0,FFFFFF
133,GJ134,4.872244 | FFC000,1,000000
134,GJ135,78.84801 | FFC000,1,000000


In [9]:
df[(df['next_color_code'] == 1) ]

,NAME,2024-04-12 00:00:00.1,next_color_code,next_color
133,GJ134,4.872244 | FFC000,1,000000
134,GJ135,78.84801 | FFC000,1,000000
